<span style="color: #2874A6;  font-weight: bold;">Unsloth Library Installation and Setup</span>

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

<span style="color: #2874A6; font-weight: bold;">Loading the LLAMA 3.1 8B Instruct Model and quantizing it to 4 bits for faster inference</span> <br>


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

<span style="color:#2874A6 ; font-weight: bold;">Checking the layers of model</span>

In [4]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

<span style="color: #2874A6; font-weight: bold;">Applying Parameter-Efficient Fine-Tuning with PEFT to only the linear layers of the model</span>

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.1.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<span style="color: #2874A6; font-weight: bold;">This code prepares and formats a dataset for training or fine-tuning a language model and converts the model to the Alpaca Prompt Template</span>

In [5]:
coupon_prompt = """Below is an instruction to generate a coupon code based on the provided brand and description. Write a response that generates an appropriate coupon code.

### Brand:
{}

### Description:
{}

### Coupon Code:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    brands = examples["Brand"]
    infos  = examples["Info"]
    coupons = examples["Discount Coupon"]
    texts = []
    for brand, info, coupon in zip(brands, infos, coupons):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = coupon_prompt.format(brand, info, coupon) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}


<span style="color: #2874A6; font-weight: bold;">We are also downloading the dataset which we uploaded to HuggingFace</span>

In [6]:
from datasets import load_dataset
dataset = load_dataset("visharxd/discount-coupons", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/66.0 [00:00<?, ?B/s]

codes.csv:   0%|          | 0.00/59.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1110 [00:00<?, ? examples/s]

Map:   0%|          | 0/1110 [00:00<?, ? examples/s]

<span style="color: #2874A6; font-weight: bold;">This code sets up a fine-tuning process for a language model using the SFTTrainer from the trl library</span>

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 10,
        max_steps = 140,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16=True,
        bf16=False,
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Map (num_proc=2):   0%|          | 0/1110 [00:00<?, ? examples/s]

<span style="color: #2874A6; font-weight: bold;"> Triggers the training process for the model using the settings and dataset specified in the SFTTrainer configuration</span>

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,110 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 140
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,3.331600
20,1.538400
30,1.379700
40,1.339900
50,1.333900
60,1.323600
70,1.325600
80,1.335300
90,1.316700
100,1.319700


<span style="color: #2874A6; font-weight: bold;"> Pushes the fine-tuned model to the Hugging Face Model Hub under a specific repository</span>

In [11]:
model.push_to_hub_gguf("visharxd/coupon-generator", tokenizer, quantization_method = "q4_k_m", token = "YOUR_HF_KEY")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.33 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 47%|████▋     | 15/32 [00:07<00:05,  3.13it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:21<00:00,  2.56s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving visharxd/coupon-generator/pytorch_model-00001-of-00004.bin...
Unsloth: Saving visharxd/coupon-generator/pytorch_model-00002-of-00004.bin...
Unsloth: Saving visharxd/coupon-generator/pytorch_model-00003-of-00004.bin...
Unsloth: Saving visharxd/coupon-generator/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at visharxd/coupon-generator into f16 GGUF format.
The output location will be /content/visharxd/coupon-generator/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: coupon-generator
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: l

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/visharxd/coupon-generator
